# Solver 파라미터 튜터리얼(Solver Parameters Tutorial)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


## solver 파라미터 설정하기(Setting solver parameters)
많은 솔버는 사용자가 매개변수를 조정할 수 있도록 합니다. `Solve()` 함수를 호출할 때, Drake는 솔버의 기본 파라미터(반복 횟수, 최적성 허용 오차 등)를 사용한다. 이러한 파라미터를 수정하는 방법은 두 가지가 있다. `MathematicalProgram::SetSolverOption`을 호출하거나 `Solve()` 함수에 `SolverOptions` 인자를 전달하는 것이다.

### Calling MathematicalProgram::SetSolverOption
`MathematicalProgram::SetSolverOption(solver_id, option_name, option_value)`를 호출하면 특정 솔버(매칭되는 solver_id를 가진)에 대한 파라미터를 설정할 수 있다. `option_name`은 해당 솔버에 특화되어 있다(예를 들어, [여기](https://coin-or.github.io/Ipopt/OPTIONS.html)에서 IPOPT 파라미터 목록을 확인할 수 있다). 이 솔버 파라미터는 `MathematicalProgram` 객체에 저장되며, 해당 솔버(매칭되는 solver_id를 가진)가 호출될 때 `Solve()` 호출에서 이 파라미터가 적용된다.

다음 코드에서 IPOPT의 옵션을 설정하는 예를 보여준다.

In [ ]:
from pydrake.solvers import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers import IpoptSolver
import numpy as np
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(x[0]**2 + x[1] ** 2)
prog.AddConstraint(x[0] + x[1] == 1)

# Set the maximum iteration for IPOPT to be 1.
# max_iter is a parameter of IPOPT solver, explained in
# https://www.coin-or.org/Ipopt/documentation/node42.html
prog.SetSolverOption(IpoptSolver().solver_id(), "max_iter", 1)
solver = IpoptSolver()
result = solver.Solve(prog, np.array([10, 1]), None)
# With fewer maximum iteration, IPOPT hasn't converged to optimality yet (The true optimal is [0.5, 0.5])
print("Success? ", result.is_success())
print(result.get_solution_result())
print("IPOPT x*= ", result.GetSolution(x))

물론 솔버의 파라미터를 설정한다고 해도 `result = Solve(prog)` 호출 시 반드시 해당 솔버가 **사용되는 것은 아니다**. Drake가 호출되는 솔버를 결정한다. Drake가 특정 문제에 가장 적합하다고 판단한 솔버를 선택한다.

다음 코드 조각에서 우리는 IPOPT를 위한 솔버의 옵션을 설정하지만, Drake는 이 특정 문제를 닫힌 형태로 해결할 수 있는 다른 솔버를 선택한다.

In [ ]:
prog.SetSolverOption(IpoptSolver().solver_id(), "max_iter", 1)
result = Solve(prog)
print(result.get_solver_id().name())

### SolverOptions을 Solve 함수로 전달(Passing a SolverOptions to Solve function)
솔버 옵션을 설정하는 또 다른 방법은 `Solve` 함수에 인자로 `SolverOptions` 객체를 전달하는 것이다. `MathematicalProgram`은 이 `SolverOptions` 객체를 저장하지 않는다.

다음 예시에서 첫 번째 `Solve` 호출은 `SolverOptions` 객체를 사용하여 IPOPT의 파라미터를 설정하고, 두 번째 `Solve` 호출은 기본 IPOPT 파라미터를 사용합니다. 따라서 두 `Solve` 호출에서 서로 다른 결과를 얻는다.

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(x[0]**2 + x[1] ** 2)
prog.AddConstraint(x[0] + x[1] == 1)

solver_options = SolverOptions()
solver_options.SetOption(IpoptSolver().solver_id(), "max_iter", 1)
solver = IpoptSolver()

# Call Solve with solver_options, IPOPT will use `max_iter` = 1
result = solver.Solve(prog, np.array([10, 1]), solver_options)
print("Success? ", result.is_success())
print(result.get_solution_result())
# Call Solve without solver_options, IPOPT will use the default options.
result = solver.Solve(prog, np.array([10, 1]), None)
print("Success? ", result.is_success())
print(result.get_solution_result())